In [113]:
import csv
import numpy as np

In [114]:
# Define object for data to be analyzed
class Data:
    def __init__(self, step, r_embb, g_embb, u_embb, r_urllc, g_urllc, u_urllc, r_miot, g_miot, u_miot, r_mmtc, g_mmtc, u_mmtc, r_voice, g_voice, u_voice, band_remaining):
        self.step = step
        self.r_embb = r_embb
        self.g_embb = g_embb
        self.u_embb = u_embb
        self.r_urllc = r_urllc
        self.g_urllc = g_urllc
        self.u_urllc = u_urllc
        self.r_miot = r_miot
        self.g_miot = g_miot
        self.u_miot = u_miot
        self.r_mmtc = r_mmtc
        self.g_mmtc = g_mmtc
        self.u_mmtc = u_mmtc
        self.r_voice = r_voice
        self.g_voice = g_voice
        self.u_voice = u_voice
        self.band_remaining = band_remaining

# Define object for finale distribution per round
class TrueDistribution:
    def __init__(self, time, true_embb, true_urllc, true_miot, true_mmtc, true_voice):
        self.time = time
        self.true_embb = true_embb
        self.true_urllc = true_urllc
        self.true_miot = true_miot
        self.true_mmtc = true_mmtc
        self.true_voice = true_voice
        
class MakeCsv:
    def __init__(self, step, r_embb, g_embb, u_embb, t_embb, r_urllc, g_urllc, u_urllc, t_urllc, r_miot, g_miot, u_miot, t_miot, r_mmtc, g_mmtc, u_mmtc, t_mmtc, r_voice, g_voice, u_voice, t_voice, br): 
        self.step = step
        self.r_embb = r_embb
        self.g_embb = g_embb
        self.u_embb = u_embb
        self.t_embb = t_embb
        self.r_urllc = r_urllc
        self.g_urllc = g_urllc
        self.u_urllc = u_urllc
        self.t_urllc = t_urllc
        self.r_miot = r_miot
        self.g_miot = g_miot
        self.u_miot = u_miot
        self.t_miot = t_miot
        self.r_mmtc = r_mmtc
        self.g_mmtc = g_mmtc
        self.u_mmtc = u_mmtc
        self.t_mmtc = t_mmtc
        self.r_voice = r_voice
        self.g_voice = g_voice
        self.u_voice = u_voice
        self.t_voice = t_voice
        self.br = br
        

In [115]:
# Parameters and functions
bs_band = 20000000000
ratio_embb = 0.65
ratio_urllc = 0.05
ratio_miot = 0.1
ratio_mmtc = 0.1
ratio_voice = 0.1

def get_ratios_br(band_available):
    ratio_embb = int(0.65 * band_available)
    ratio_urllc = int(0.05 * band_available)
    ratio_miot = int(0.1 * band_available)
    ratio_mmtc = int(0.1 * band_available)
    ratio_voice = int(0.1 * band_available)
        
    #TODO: better way to distributed band when some goes to 0
        
    ratios  = {
        'URLLC': ratio_urllc,
        'eMBB': ratio_embb,
        'mMTC': ratio_mmtc,
        'voice': ratio_voice,
        'MIoT': ratio_miot
    }
    
    return ratios

In [116]:
# Read data and put them in a list to easy manage
with open('input.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    i = 0
    data_list = []
    
    for row in csv_reader:
        embb_g =int(row['embb_rate'])
        embb_u = int(row['embb_users'])
        embb_r = int(row['embb_req'])
        urllc_g = int(row['urllc_rate'])
        urllc_u = int(row['urllc_users'])
        urllc_r = int(row['urllc_req'])
        miot_g = int(row['miot_rate'])
        miot_u = int(row['miot_users'])
        miot_r = int(row['miot_req'])
        mmtc_g = int(row['mmtc_rate'])
        mmtc_u = int(row['mmtc_users'])
        mmtc_r = int(row['mmtc_req']) 
        voice_g = int(row['voice_rate'])
        voice_u = int(row['voice_users'])
        voice_r = int(row['voice_req'])
        band_rem = bs_band - (embb_g + urllc_g + miot_g + mmtc_g + voice_g)
        
        data_step = Data(i, embb_r, embb_g, embb_u, urllc_r, urllc_g, urllc_u, miot_r, miot_g, miot_u, mmtc_r, mmtc_g, mmtc_u, voice_r, voice_g, voice_u, band_rem)
        i += 1
        
        data_list.append(data_step)
        

In [117]:
# Compute new distribution
i = 0
true_ratios = []

for item in data_list:
    
    print('STEP', i)
    
    br = item.band_remaining
    total_req = item.r_embb  + item.r_urllc + item.r_miot + item.r_mmtc + item.r_voice
    embb_g_temp, urllc_g_temp, miot_g_temp, mmtc_g_temp, voice_g_temp = item.g_embb, item.g_urllc, item.g_miot, item.g_mmtc, item.g_voice
    new_g_embb, new_g_urllc, new_g_miot, new_g_mmtc, new_g_voice = 0, 0, 0, 0, 0
    
    while br >= 2 and total_req != new_g_embb:
        available_per_slice = get_ratios_br(br)
        
        if available_per_slice['eMBB'] != 0:
            delta_embb = item.r_embb - embb_g_temp - available_per_slice['eMBB']
            if delta_embb > 0:
                embb_g_temp = embb_g_temp + available_per_slice['eMBB']
                new_g_embb = embb_g_temp
                br = br - available_per_slice['eMBB']
            elif delta_embb == 0:
                new_g_embb = item.r_embb 
                br = br - available_per_slice['eMBB']
            elif delta_embb < 0:
                new_g_embb = item.r_embb
                br = br - available_per_slice['eMBB'] + abs(delta_embb)
        
        if available_per_slice['URLLC'] != 0:
            delta_urllc = item.r_urllc - urllc_g_temp - available_per_slice['URLLC']
            if delta_urllc > 0:
                urllc_g_temp = urllc_g_temp + available_per_slice['URLLC']
                new_g_urllc = urllc_g_temp
                br = br - available_per_slice['URLLC']
            elif delta_urllc == 0:
                new_g_urllc = item.r_urllc
                br = br - available_per_slice['URLLC']
            elif delta_urllc < 0:
                new_g_urllc = item.r_urllc
                br = br - available_per_slice['URLLC'] + abs(delta_urllc)
        
        if available_per_slice['MIoT'] != 0:
            delta_miot = item.r_miot - miot_g_temp - available_per_slice['MIoT']
            if delta_miot > 0:
                miot_g_temp = miot_g_temp + available_per_slice['MIoT']
                new_g_miot = miot_g_temp
                br = br - available_per_slice['MIoT']
            elif delta_miot == 0:
                new_g_miot = item.r_miot
                br = br - available_per_slice['MIoT']
            elif delta_miot < 0:
                new_g_miot = item.r_miot
                br = br - available_per_slice['MIoT'] + abs(delta_miot)
                
        if available_per_slice['mMTC'] != 0:
            delta_mmtc = item.r_mmtc - mmtc_g_temp - available_per_slice['mMTC']
            if delta_mmtc > 0:
                mmtc_g_temp = mmtc_g_temp + available_per_slice['mMTC']
                new_g_mmtc = mmtc_g_temp
                br = br - available_per_slice['mMTC']
            elif delta_mmtc == 0:
                new_g_mmtc = item.r_mmtc
                br = br - available_per_slice['mMTC']
            elif delta_mmtc < 0:
                new_g_mmtc = item.r_mmtc
                br = br - available_per_slice['mMTC'] + abs(delta_mmtc)
        
        if available_per_slice['voice'] != 0:
            delta_voice = item.r_voice - voice_g_temp - available_per_slice['voice']
            if delta_voice > 0:
                voice_g_temp = voice_g_temp + available_per_slice['voice']
                new_g_voice = voice_g_temp
                br = br - available_per_slice['voice']
            elif delta_voice == 0:
                new_g_voice = item.r_voice
                br = br - available_per_slice['voice']
            elif delta_voice < 0:
                new_g_voice = item.r_voice
                br = br - available_per_slice['voice'] + abs(delta_voice)
                         
    true = TrueDistribution(i, new_g_embb, new_g_urllc, new_g_miot, new_g_mmtc, new_g_voice)
    true_ratios.append(true)
    i += 1   
    

STEP 0
STEP 1
STEP 2
STEP 3
STEP 4
STEP 5
STEP 6
STEP 7
STEP 8
STEP 9
STEP 10
STEP 11
STEP 12
STEP 13
STEP 14
STEP 15
STEP 16
STEP 17
STEP 18
STEP 19
STEP 20
STEP 21
STEP 22
STEP 23
STEP 24
STEP 25
STEP 26
STEP 27
STEP 28
STEP 29
STEP 30
STEP 31
STEP 32
STEP 33
STEP 34
STEP 35
STEP 36
STEP 37
STEP 38
STEP 39


In [118]:
# Debug
# NEW bps VALUES

for item in true_ratios:
    embb += item.true_embb
    urllc += item.true_urllc
    miot += item.true_miot
    mmtc += item.true_mmtc
    voice += item.true_voice
    count += 1
embb = int((embb/count)/mega)
urllc = int((urllc/count)/mega)
miot = int((miot/count)/mega)
mmtc = int((mmtc/count)/mega)
voice = int((voice/count)/mega)

# OLD bps VALUES

for item in data_list:
    old_embb += item.g_embb
    old_urllc += item.g_urllc
    old_miot += item.g_miot
    old_mmtc += item.g_mmtc
    old_voice += item.g_voice
    count += 1

old_embb = int((old_embb/count)/mega)
old_urllc = int((old_urllc/count)/mega)
old_miot = int((old_miot/count)/mega)
old_mmtc = int((old_mmtc/count)/mega)


# RESULTS 

old_voice = int((old_voice/count)/mega)
print(f'(embb) before VS now: {old_embb} Mbps, {embb} Mbps')
print(f'(urllc) before VS now: {old_urllc} Mbps, {urllc} Mbps')
print(f'(miot) before VS now: {old_miot} Mbps, {miot} Mbps')
print(f'(mmtc) before VS now: {old_mmtc} Mbps, {mmtc} Mbps')
print(f'(voice) before VS now: {old_voice} Mbps, {voice} Mbps')


(embb) before VS now: 562 Mbps, 1218 Mbps
(urllc) before VS now: 13 Mbps, 14 Mbps
(miot) before VS now: 9 Mbps, 10 Mbps
(mmtc) before VS now: 11 Mbps, 12 Mbps
(voice) before VS now: 5 Mbps, 14 Mbps


In [120]:
# MAKE OUTPUT.CSV

step = 0
row = []

for data,ratio in zip(data_list, true_ratios):
    step = data.step
    r_embb = data.r_embb
    g_embb = data.g_embb
    u_embb = data.u_embb
    r_urllc = data.r_urllc
    g_urllc = data.g_urllc
    u_urllc = data.u_urllc
    r_miot = data.r_miot
    g_miot = data.g_miot
    u_miot = data.u_miot
    r_mmtc = data.r_mmtc
    g_mmtc = data.g_mmtc
    u_mmtc = data.u_mmtc
    r_voice = data.r_voice
    g_voice = data.g_voice
    u_voice = data.u_voice
    t_embb = ratio.true_embb
    t_urllc = ratio.true_urllc
    t_miot = ratio.true_miot
    t_mmtc = ratio.true_mmtc
    t_voice = ratio.true_voice
    br = data.band_remaining
    data = MakeCsv(step, r_embb, g_embb, u_embb, t_embb, r_urllc, g_urllc, u_urllc, t_urllc, r_miot, g_miot, u_miot, t_miot, r_mmtc, g_mmtc, u_mmtc, t_mmtc, r_voice, g_voice, u_voice, t_voice, br) 
    row.append(data)
    
    
with open('output.csv', mode='w') as csv_file:
    fieldnames = ['round', 'embb_req', 'embb_giv', 'embb_user', 'embb_true', 'urllc_req', 'urllc_giv', 'urllc_user', 'urllc_true','miot_req', 'miot_giv', 'miot_user', 'miot_true','mmtc_req', 'mmtc_giv', 'mmtc_user', 'mmtc_true','voice_req', 'voice_giv', 'voice_user', 'voice_true', 'band_rem']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for item in row:
        writer.writerow({'round':item.step, 'embb_req':item.r_embb, 'embb_giv':item.g_embb, 'embb_user':item.u_embb, 'embb_true':item.t_embb, 'urllc_req':item.r_urllc, 'urllc_giv':item.g_urllc, 'urllc_user':item.u_urllc, 'urllc_true':item.t_urllc,'miot_req':item.r_miot, 'miot_giv':item.g_miot, 'miot_user':item.u_miot, 'miot_true':item.t_miot,'mmtc_req':item.r_mmtc, 'mmtc_giv':item.g_mmtc, 'mmtc_user':item.u_mmtc, 'mmtc_true':item.t_mmtc,'voice_req':item.r_voice, 'voice_giv':item.g_voice, 'voice_user':item.u_voice, 'voice_true':item.t_voice, 'band_rem':item.br})
